I was unable to find any large dataset which contained user emails of the type I wanted.
Fitting any clustering algorithm on this dataset was producing satisfactory but not good results
hence, I discarded the idea.
however this notebook still contains much of the work which I tried.

In [ ]:
import re

import joblib
import pandas as pd
from sklearn.impute import SimpleImputer
import dill

mails_df = pd.read_csv("../data/training_data.csv", sep='~', index_col=0)
imputer = SimpleImputer(strategy='constant', fill_value='')
mails_df = pd.DataFrame(imputer.fit_transform(mails_df), columns=mails_df.columns)
mails_df.iloc[:10]

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

In [53]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    processed_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return processed_text


def preprocess_sender(address):
    address = address.lower()
    address = re.sub('[.]ac[.]in|[.]com', '', address)
    address = re.sub('@|[.]', ' ', address)
    return address

subject_tfidf = TfidfVectorizer(preprocessor=preprocess_text, min_df=0.01)
body_tfidf = TfidfVectorizer(preprocessor=preprocess_text, max_df=0.8, min_df=0.05)
sender_tfidf = TfidfVectorizer(preprocessor=preprocess_sender)

subject_vectors = subject_tfidf.fit_transform(mails_df['subject'])
body_vectors = body_tfidf.fit_transform(mails_df['body'])
sender_vectors = sender_tfidf.fit_transform(mails_df['sender'])

In [54]:
from scipy.sparse import hstack

feature_matrix = hstack((subject_vectors, body_vectors, sender_vectors))

In [55]:
type(feature_matrix)

scipy.sparse._csr.csr_matrix

In [56]:
preprocess_sender('hello@cs.iitr.ac.in')

'hello cs iitr'

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

with open('../data/label_dict.json', 'r') as file:
    all_labels = json.load(file)
label_list = [key for key in all_labels.keys() if re.match('Label_[0-9]', key)]
mlb = MultiLabelBinarizer(classes=label_list)
labels_array = [list(st.split(',')) for st in mails_df['labels']]
mlb.fit(label_list)
labels = mlb.transform(labels_array)

In [58]:
X = feature_matrix
y = labels
print(X.shape, y.shape)

(1198, 601) (1198, 21)


In [ ]:
from sklearn.cluster import KMeans

k = 25
model =  KMeans(n_clusters=25, n_init=10, random_state=42)
labels = model.fit_predict(X.toarray())
labels

In [60]:
mails_df['label'] = pd.Series(labels)

In [61]:
new_df = mails_df.loc[:,['id', 'subject', 'label', 'labels']]

vectorizers = [sender_tfidf, body_tfidf, subject_tfidf]
dill.dump(vectorizers, open('../data/TfidfVectorizers.pkl', 'wb'))

vectorizers = dill.load(open('../data/TfidfVectorizers.pkl', 'rb'))

In [62]:
new_df.reset_index(inplace=True)

In [63]:
new_df.sort_values(by=['label', 'index'], inplace=True)

In [ ]:
new_df

In [ ]:
new_df.loc[new_df['label'] == 1]

In [66]:
def write_label_names(label_id_series: pd.Series, exc_list=[]) -> pd.Series:
    """
    :param exc_list: a list of label_ids among the default ones which are required
    will convert comma separated label_ids into label_names for a series of strings
    """
    labels_dict = all_labels

    def label_filter(label_id_st):
        label_names = []
        for label_id in label_id_st:
            if label_id == labels_dict[label_id] and label_id not in exc_list:
                label_id_st.remove(label_id)
            else:
                label_names.append(labels_dict[label_id])

        return ','.join(label_names)

    return label_id_series.str.split(',').apply(label_filter)

In [67]:
new_df['label_list'] = write_label_names(new_df['labels'])

In [68]:
for i in range(25):
    print(new_df.loc[new_df['label'] == i].label_list.value_counts())
    print()

                  29
Guest Lectures     3
Name: label_list, dtype: int64

                       34
bhawan notifs           2
jobs and internship     2
Name: label_list, dtype: int64

                                     371
jobs and internship                   25
IMP                                   11
academics,ETE                          9
Class materials,academics              8
bhawan notifs                          4
institute updates                      3
Guest Lectures                         3
Techincal Clubs                        3
OTP                                    2
academics                              2
SCSP                                   2
MTE,academics                          2
Club Recruitments,Techincal Clubs      1
Club Recruitments                      1
MTE,academics,ETE                      1
hackathons                             1
treks                                  1
Class materials                        1
AAO                                  

In [ ]:
model.transform(X)

In [ ]:
from sklearn.mixture import GaussianMixture

k = 25
model =  GaussianMixture(n_components=25, n_init=10)
labels = model.fit_predict(X.toarray())
labels

In [71]:
model.converged_

True

In [72]:
model.n_iter_

2

In [ ]:
mails_df['label'] = pd.Series(labels)
new_df = mails_df.loc[:, ['id', 'subject', 'label', 'labels']]
new_df.reset_index(inplace=True)
new_df.sort_values(by=['label', 'index'], inplace=True)
new_df

In [ ]:
new_df.loc[new_df['label'] == 1]

In [75]:
def write_label_names(label_id_series: pd.Series, exc_list=[]) -> pd.Series:
    """
    :param exc_list: a list of label_ids among the default ones which are required
    will convert comma separated label_ids into label_names for a series of strings
    """
    labels_dict = all_labels

    def label_filter(label_id_st):
        label_names = []
        for label_id in label_id_st:
            if label_id == labels_dict[label_id] and label_id not in exc_list:
                label_id_st.remove(label_id)
            else:
                label_names.append(labels_dict[label_id])

        return ','.join(label_names)

    return label_id_series.str.split(',').apply(label_filter)


new_df['label_list'] = write_label_names(new_df['labels'])
for i in range(25):
    print(new_df.loc[new_df['label'] == i].label_list.value_counts())
    print()

                  7
commercial        5
commercial,OTP    3
Name: label_list, dtype: int64

                                     64
Techincal Clubs                      15
Club Recruitments,Techincal Clubs    14
hackathons                            3
Club Recruitments                     2
IMP,Techincal Clubs                   1
IMP                                   1
Name: label_list, dtype: int64

AAO info    34
AAO          1
Name: label_list, dtype: int64

academics,ETE                9
MTE,academics                9
Class materials,academics    8
Quiz,academics               7
MTE,academics,ETE            3
Quiz                         2
ETE                          1
MTE                          1
MTE,Quiz,academics           1
Class materials              1
Name: label_list, dtype: int64

                       43
bhawan notifs           3
bhawan notifs,IMP       2
jobs and internship     2
ETE                     1
Name: label_list, dtype: int64

                       72
bhaw

It can be clearly inferred from the last outputs that the clustering is certainly giving satisfactory results but